In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')
test = test_data.copy()
y = data['Survived']
X = data.drop(['Cabin','Name','Survived','Ticket','PassengerId'],axis=1)
test.drop(['Cabin','Name','Ticket','PassengerId'],axis=1,inplace=True)
data.columns
#missing_values = (data.isnull().sum())
#print(missing_values[missing_values > 0])
#print(missing_values)


In [ ]:
y.isnull().sum()

In [ ]:
# One Hot Encoding
one_hot_encoded_data = pd.get_dummies(X)
test = pd.get_dummies(test)
X = one_hot_encoded_data

In [ ]:
# Imputing Data
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()

imputed_train_data = pd.DataFrame(my_imputer.fit_transform(X))
imputed_train_data.columns = X.columns

imputed_test_data = pd.DataFrame(my_imputer.fit_transform(test))
imputed_test_data.columns = test.columns


In [ ]:
# Dividing Train Data into test and train to later check the accuracy of the model
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(imputed_train_data,y,test_size = 0.25,random_state = 10)


In [ ]:
# Checking Accuracy of the model and tuning parameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
max_leaf_nodes = [5,10,50,100,200,500]
for leaf_node in max_leaf_nodes:
    model = RandomForestClassifier(n_estimators=100,criterion='entropy',random_state=10,)
    model.fit(X_train,y_train)
    print(leaf_node,' : ',accuracy_score(y_test,model.predict(X_test)))

In [ ]:
# Using XGBoost Model
import xgboost as xgb
from sklearn.metrics import accuracy_score
model2 = xgb.XGBClassifier(max_depth=3,n_estimators=300,learning_rate=0.05)
model2.fit(X_train,y_train)
model2.predict(X_test)
print(accuracy_score(y_test,model2.predict(X_test)))

In [ ]:
# Testing the model
predictions = model2.predict(imputed_test_data)
# Making SUbmission
my_submission = pd.DataFrame({'PassengerId':test_data.PassengerId,'Survived':predictions})
my_submission.to_csv('submission3.csv',index=False)
predictions